Homework 4: Sentiment Analysis - Task 0, Task 1, Task 5 (all primarily written tasks)
----

The following instructions are only written in this notebook but apply to all notebooks and `.py` files you submit for this homework.

Due date: February 28th, 2024

Points: 
- Task 0: 5 points
- Task 1: 10 points
- Task 2: 30 points
- Task 3: 20 points
- Task 4: 20 points
- Task 5: 15 points

Goals:
- understand the difficulties of counting and probablities in NLP applications
- work with real world data to build a functioning language model
- stress test your model (to some extent)

Complete in groups of: __two (pairs)__. If you prefer to work on your own, you may, but be aware that this homework has been designed as a partner project.

Allowed python modules:
- `numpy`, `matplotlib`, `keras`, `pytorch`, `nltk`, `pandas`, `sci-kit learn` (`sklearn`), `seaborn`, and all built-in python libraries (e.g. `math` and `string`)
- if you would like to use a library not on this list, post on piazza to request permission
- all *necessary* imports have been included for you (all imports that we used in our solution)

Instructions:
- Complete outlined problems in this notebook. 
- When you have finished, __clear the kernel__ and __run__ your notebook "fresh" from top to bottom. Ensure that there are __no errors__. 
    - If a problem asks for you to write code that does result in an error (as in, the answer to the problem is an error), leave the code in your notebook but commented out so that running from top to bottom does not result in any errors.
- Double check that you have completed Task 0.
- Submit your work on Gradescope.
- Double check that your submission on Gradescope looks like you believe it should __and__ that all partners are included (for partner work).


Task 0: Name, References, Reflection (5 points)
---

Names
----
Names: Kaan Tural, Arinjay Singh

References
---
List the resources you consulted to complete this homework here. Write one sentence per resource about what it provided to you. If you consulted no references to complete your assignment, write a brief sentence stating that this is the case and why it was the case for you.

(Example)
- https://docs.python.org/3/tutorial/datastructures.html
    - Read about the the basics and syntax for data structures in python.


Reflection
----
Answer the following questions __after__ you complete this assignment (no more than 1 sentence per question required, this section is graded on completion):

1. Does this work reflect your best effort?
2. What was/were the most challenging part(s) of the assignment?
3. If you want feedback, what function(s) or problem(s) would you like feedback on and why?
4. Briefly reflect on how your partnership functioned--who did which tasks, how was the workload on each of you individually as compared to the previous homeworks, etc.

Task 1: Provided Data Write-Up (10 points)
---

Every time you use a data set in an NLP application (or in any software application), you should be able to answer a set of questions about that data. Answer these now. Default to no more than 1 sentence per question needed. If more explanation is necessary, do give it.

This is about the __provided__ movie review data set.

1. Where did you get the data from? The provided dataset(s) were sub-sampled from https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews 
2. (1 pt) How was the data collected (where did the people acquiring the data get it from and how)?

It is not mentioned on the website, the assumption can be made they looked at a popular movie review site such as IMDB though and collected ones with certain ratings or responses that may make them "highly polar" possibly ratings that are the complete inverse of what the true movie's rating was using a scraper.

3. (2 pts) How large is the dataset (answer for both the train and the dev set, separately)? (# reviews, # tokens in both the train and dev sets)

There were 25,000 reviews in the training data and 25,000 reviews in the dev/test set. But within the data that we have in the .txt files, it is 1600 reviews in training set and 200 views in the data set. 369136 tokens in training, 47158 in dev.

4. (1 pt) What is your data? (i.e. newswire, tweets, books, blogs, etc)

The data is made up of movie reviews that are considered highly polar.

5. (1 pt) Who produced the data? (who were the authors of the text? Your answer might be a specific person or a particular group of people)

The people at stanford, it can likely be assumed to be Andrew Maas as they are the only name mentioned. It is implied they used a webscaper on IMDB in the discussions.

6. (2 pts) What is the distribution of labels in the data (answer for both the train and the dev set, separately)?

It appears the distribution between positive and negative is 50:50 as well. With slightly more positive reviews as it is titled as the "most common". The actual numbers are 1 - 50.25  and 0 - 49.75 which means that there are .5% more positive reviews.

7. (2 pts) How large is the vocabulary (answer for both the train and the dev set, separately)?

Vocabulary size in training set: 47638
Vocabulary size in dev set: 11709

8. (1 pt) How big is the overlap between the vocabulary for the train and dev set?

Vocabulary overlap between training and dev sets: 7245

In [ ]:
# our utility functions
# RESTART your jupyter notebook kernel if you make changes to this file
import sentiment_utils as sutils



C:\Users\Isha\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Isha\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Isha\anaconda3\lib\site-packages\numpy\.libs\libopenblas.xwydx2ikjw2nmtwsfyngfuwkqu3lytcz.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Isha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# Feel free to write code to help answer the above questions

# Used GPT to generate the ways of calculating the above questions
import pandas as pd

train_path = 'test_dev_data/movie_reviews_train.txt'
dev_path = 'test_dev_data/movie_reviews_dev.txt'

train_df = pd.read_csv(train_path, sep='\t', header=None, names=['Review', 'Label'])
dev_df = pd.read_csv(dev_path, sep='\t', header=None, names=['Review', 'Label'])

# Count the number of reviews in each set
num_reviews_train = len(train_df)
num_reviews_dev = len(dev_df)

# Print out the counts
print(f"Number of reviews in training set: {num_reviews_train}")
print(f"Number of reviews in dev set: {num_reviews_dev}")

train_df['Token Count'] = train_df['Review'].apply(lambda x: len(x.split()))
dev_df['Token Count'] = dev_df['Review'].apply(lambda x: len(x.split()))

# Total number of tokens in each set
total_tokens_train = train_df['Token Count'].sum()
total_tokens_dev = dev_df['Token Count'].sum()

# Distribution of labels in both sets
label_distribution_train = train_df['Label'].value_counts(normalize=True) * 100
label_distribution_dev = dev_df['Label'].value_counts(normalize=True) * 100

# Calculate vocabulary size
# Combine all reviews into a single string for each set, then split into tokens and convert to a set to remove duplicates
vocab_train = set(' '.join(train_df['Review']).split())
vocab_dev = set(' '.join(dev_df['Review']).split())

# Vocabulary sizes
vocab_size_train = len(vocab_train)
vocab_size_dev = len(vocab_dev)

# Vocabulary overlap
vocab_overlap = len(vocab_train.intersection(vocab_dev))

# Printing the results
print(f"Total number of tokens in training set: {total_tokens_train}")
print(f"Total number of tokens in dev set: {total_tokens_dev}")
print(f"Label distribution in training set (%):\n{label_distribution_train}")
print(f"Label distribution in dev set (%):\n{label_distribution_dev}")
print(f"Vocabulary size in training set: {vocab_size_train}")
print(f"Vocabulary size in dev set: {vocab_size_dev}")
print(f"Vocabulary overlap between training and dev sets: {vocab_overlap}")

Number of reviews in training set: 1600
Number of reviews in dev set: 200
Total number of tokens in training set: 369136
Total number of tokens in dev set: 47158
Label distribution in training set (%):
Label
1    50.25
0    49.75
Name: proportion, dtype: float64
Label distribution in dev set (%):
Label
1    52.5
0    47.5
Name: proportion, dtype: float64
Vocabulary size in training set: 47638
Vocabulary size in dev set: 11709
Vocabulary overlap between training and dev sets: 7245


Task 5: Model Evaluation (15 points)
---
Save your three graph files for the __best__ configurations that you found with your models using the `plt.savefig(filename)` command. The `bbox_inches` optional parameter will help you control how much whitespace outside of the graph is in your resulting image.

__NOTE:__ Run each notebook containing a classifier 3 times, resulting in __NINE__ saved graphs (don't just overwrite your previous ones).

You will turn in all of these files.

10 points in this section are allocated for having all nine graphs legible, properly labeled, and present.




1. (1 pt) When using __10%__ of your data, which model had the highest f1 score?
2. (1 pt) Which classifier had the most __consistent__ performance (that is, which classifier had the least variation across all three graphs you have for it -- no need to mathematically calculate this, you can just look at the graphs)?
3. (1 pt) For each model, what percentage of training data resulted in the highest f1 score?
    1. Naive Bayes:
    2. Logistic Regression:
    3. Neural Net:
4. (2 pts) Which model, if any, appeared to overfit the training data the most? Why?
